# PromptTemplate 
- 템플릿화된 프롬프트를 만들 수 있게 해주는 클래스
- PromptTemplate() : 변수가 포함된 템플릿 객체를 만들 수 있게 해준다.
    - template: 프롬프트 문자열. {country}는 나중에 값으로 대체될 변수.
    - input_variables: 템플릿에서 사용할 변수명을 리스트로 지정해야 함.
- invoke(): 템플릿에 실제 값을 넣어서, 사용할 문장을 만들어 줌.
- 완성된 프롬프트를 LLM에 전달하여 응답을 받을 수 있다.


In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-04T09:04:06.2541099Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3236246500, 'load_duration': 2695830200, 'prompt_eval_count': 32, 'prompt_eval_duration': 241230600, 'eval_count': 8, 'eval_duration': 298409100, 'model_name': 'llama3.2:1b'}, id='run--3dbba2d9-cc40-4892-add7-73b142650c6e-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="What is te capital of {country}?",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})

print(prompt)

llm.invoke(prompt)


text='What is te capital of France?'


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-04T09:26:34.8439501Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1847780500, 'load_duration': 1400240100, 'prompt_eval_count': 32, 'prompt_eval_duration': 209836400, 'eval_count': 8, 'eval_duration': 236602600, 'model_name': 'llama3.2:1b'}, id='run--4e26fbd5-21b2-4ffd-bfde-0eb4f75aa4c1-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

## Langchain 의 메시지 클래스
1.HumanMessage
- BaseMessage를 상속받는다.
- 사용자가 LLM에게 보내는 입력 메시지를 의미한다.
- 대화형 시스템에서 사용자의 질문이나 지시를 표현할 때 사용된다.

2.SystemMessage
- BaseMessage를 상속받는다.
- LLM의 동작 방식과 응답 스타일을 정의하는 지시사항을 담는다.
- 대화 시작 시 LLM에게 역할, 제약조건, 목적 등을 알려줄 때 사용된다.

3.AiMessage
- BaseMessage를 상속받는다.
- LLM이 생성한 응답을 나타낸다.
- 대화 기록을 저장하거나 이전 AI 응답을 참조할 때 사용된다.

이 메시지들은 리스트로 감싸주어야 한다.

아래와 같이 작성해서 실행하면 에러가 발생한다. 
- ValueError: Invalid input type <class 'langchain_core.messages.human.HumanMessage'>. Must be a PromptValue, str, or list of BaseMessages.

In [10]:
from langchain_core.messages import HumanMessage

llm.invoke(HumanMessage(content="What is the capital of France?"))

ValueError: Invalid input type <class 'langchain_core.messages.human.HumanMessage'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="What is the capital of France?")])

## message_list 를 만들고 그 안에 여러가지 메시지들을 만들어서 한번에 llm에 전달해보자.
- 아래 예제를 보면 마치 LLM과 이전에 대화를 나눈것과 같은 느낌이 들게 한다.
- 마치 대화를 이전에 나눴던 것처럼 LLM을 속이는 것이다.
- 실제로 이런것을 fewshots(예제) 이라고 하는데 이렇게 에제를 넣어주는것이 LLM의 답변 생성 정확도에 긍정적인 영향을 끼친다는 논문이 있다.
- chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/pdf/2005.14165

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

message_list = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is Paris."),
    HumanMessage(content="What is the capital of Germany?"),
]

llm.invoke(message_list)

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-04T09:37:47.1031037Z', 'done': True, 'done_reason': 'stop', 'total_duration': 930326300, 'load_duration': 17804100, 'prompt_eval_count': 62, 'prompt_eval_duration': 564410600, 'eval_count': 8, 'eval_duration': 346516300, 'model_name': 'llama3.2:1b'}, id='run--8506dbaf-9ccb-4f2b-be61-4f350348653b-0', usage_metadata={'input_tokens': 62, 'output_tokens': 8, 'total_tokens': 70})

# ChatPromptTemplate
ChatPromptTemplate은 LangChain에서 채팅 기반 언어 모델을 위한 프롬프트 템플릿이다.
- 여러 메시지 유형(SystemMessage, HumanMessage, AIMessage)을 포함하는 템플릿을 생성할 수 있다.
- 각 메시지에 변수 삽입을 할 수 있게 지원한다.
- 채팅 모델이 기대하는 메시지 시퀀스 형식으로 구성한다.


주의할 점
- 메시지는 (역할, 내용) 형식의 튜플로 작성해야 한다.

위 PromptTemplate 에서도 사용할 수 있는데 왜 이 방식을 사용하나?
- LCEL에서 이 방식이 더 유용하다.

In [13]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
    ]
)

chat_prompt = chat_prompt_template.invoke({"input": "France"})

llm.invoke(chat_prompt)


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-04T09:53:03.2824316Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1904374100, 'load_duration': 1372310400, 'prompt_eval_count': 38, 'prompt_eval_duration': 269322400, 'eval_count': 8, 'eval_duration': 261478900, 'model_name': 'llama3.2:1b'}, id='run--1acf7f82-4f51-43ee-a3ca-259893e4bae9-0', usage_metadata={'input_tokens': 38, 'output_tokens': 8, 'total_tokens': 46})